In [1]:
from resources.runmanager import RunManager
from resources.runbuilder import RunBuilder
from resources.network import Network
from collections import OrderedDict
import torch
from collections import namedtuple
from itertools import product
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

In [2]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

In [3]:
def save_model(run, network):
    # Specify a path
    PATH = f'state_dict_model-{run}.pt'

    # Save
    torch.save(network.state_dict(), PATH)

In [4]:
# params = OrderedDict(
#     lr=[.01, .001, .0001, .00001]
#     , batch_size=[1000, 2000]
#     , shuffle=[True, False]
#     , device = ['cpu', 'cuda']
# )
params = OrderedDict(
    lr=[.01]
    , batch_size=[1000]
    , shuffle=[True]
    , device = ['cuda']
)
m = RunManager()

for run in RunBuilder.get_runs(params):
    print(type(run))
    print("IW")
    device=torch.device(run.device)
    network=Network().to(device)
    loader=torch.utils.data.DataLoader(train_set, batch_size=run.batch_size, shuffle=run.shuffle)
    optimizer=optim.Adam(network.parameters(), lr=run.lr)

    m.begin_run(run, network, loader)

    for epoch in range(5):
        m.begin_epoch()
        for batch in loader:
            images = batch[0].to(device)
            labels = batch[1].to(device)
            preds = network(images) # Pass Batch
            loss = F.cross_entropy(preds, labels) # Calculate Loss

            optimizer.zero_grad()
            loss.backward() # Calculate Gradients
            optimizer.step() # Update Weights

            m.track_loss(loss, batch)
            m.track_num_correct(preds, labels)
        m.end_epoch()
    m.end_run()
    save_model(run, network)
m.save('results')


,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,shuffle,device
0,1,1,0.960893,0.636417,5.544725,6.262245,0.01,1000,True,cuda
1,1,2,0.534218,0.790950,5.822981,12.207196,0.01,1000,True,cuda
2,1,3,0.433654,0.839567,5.632792,17.976825,0.01,1000,True,cuda
3,1,4,0.378645,0.860400,5.643548,23.722756,0.01,1000,True,cuda
4,1,5,0.346931,0.871583,5.554591,29.405211,0.01,1000,True,cuda
